## Install OpenAI Library

In [18]:
pip install openai


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install python-dotenv


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Import Library

In [20]:
import openai
import json
from dotenv import dotenv_values
config = dotenv_values(".env")

## Add API key

In [21]:
openai.api_key  = config["OPENAI_API_KEY"]

## Declare Functions

In [22]:
from IPython.display import Markdown, display

def display_colors(colors, names):
    display(Markdown(" ".join(
        f'<div style="color: {color}">{name}: {chr(9608) * 5}</div>'
        for color, name in zip(colors, names)
    )))

### 1st Prompt

In [112]:
def get_and_render_prompt(text):
    message=f"""
    You are a color palette generating assistant that responds to text prompt for color palettes.
    You should generate color palettes that fit the theme, mood, tone, or instructions in the prompt.
    
    Key Guidelines:
    - Avoid duplicating colors code and name within the same palette.
    - Keep the palettes within a range of 2 to 8 colors.
    - Use a JSON array to represent the color palettes, with each color code in hexadecimal format.
    
    Examples:
    - Q: Convert the following verbal description of a color palette into a list of colors: google brand
      A: ["#4285f4", "#34a853", "#fbbc05", "#ea4335"]

    - Q: Convert the following verbal description of a color palette into a list of colors: midnight pastels
      A: ["#B27DFA", "#6C5DC7", "#3E4AB0", "#2C3E8C", "#182F61", "#0B1E3E", "#080620"]

    Desired Format:
      - a JSON array of hexadecimal color codes

    Prompt: {text}
    Result:
    """

    completion = openai.Completion.create(
            model="text-davinci-003",
            prompt=message,
            max_tokens=100,
            temperature=0.8,
        )
    print(completion.choices[0].text)
    print("-----")
    print(completion)

    colors = json.loads(completion.choices[0].text.strip())
    display_colors(colors)

### 2nd Prompt (GPT-3)

In [26]:
def get_and_render_prompt(text):
    message=f"""
    Color Palette Generator

    Generate color palettes that match the given theme, mood, or instructions.

    Instructions:
    - Provide a verbal description of the color palette you want.
    - Avoid duplicating colors within the same palette.
    - Keep the palette size between 2 to 8 colors.

    Example Usage:
    - Instruction: Generate a color palette for a Google brand.
    - Output:"["#4285f4", "#34a853", "#fbbc05", "#ea4335"],["Google Blue", "Google Green", "Google Yellow", "Google Red"]"

    - Instruction: Generate a color palette for ocean pastels color palette.
    - Output:"["#83ADFC", "#53CBED", "#22D9DB", "#14D59C", "#0EC36F", "#2EAC8D", "#2A8FBC"], ["Sky Blue", "Light Blue", "Turquoise", "Mint Green", "Emerald Green", "Teal", "Navy Blue"]"

    Desired Response JSON Format:"["#color1", "#color2", ...], ["name1", "name2", ...]"

    Instruction: Generate a color palette for {text}

    Result:
    """

    completion = openai.Completion.create(
            model="text-davinci-003",
            prompt=message,
            max_tokens=256,
            top_p=0.5,
            frequency_penalty=0,
            presence_penalty=0
        )
    print(completion.choices[0].text)
    print("-----")
    print(completion)

    result = completion.choices[0].text
    
    colors, names = result.split("], [")

    colors = f"""{colors}]"""
    names = f"""[{names}"""
    
    print(colors)
    print(names)
    
    colors = json.loads(colors)
    names = json.loads(names)
    
    display_colors(colors, names)

### 3rd Prompt (GPT-3.5)

In [30]:
def get_and_render_prompt_chat(text):
    messages = [
        {
            "role": "system",
            "content": "You're Color Palette Generator. You'll generate color palettes that match the given theme, mood, or instructions. You'll avoid duplicating colors within the same palette. You'll keep the palette size between 2 to 8 colors."
        },
        {
            "role": "user",
            "content": "Generate a color palette for a Google brand."
        },
        {
            "role": "assistant",
            "content": '["#4285f4", "#34a853", "#fbbc05", "#ea4335"], ["Google Blue", "Google Green", "Google Yellow", "Google Red"]'
        },
        {
            "role": "user",
            "content": "Generate a color palette for ocean pastels color palette."
        },
        {
            "role": "assistant",
            "content": '["#83ADFC", "#53CBED", "#22D9DB", "#14D59C", "#0EC36F", "#2EAC8D", "#2A8FBC"], ["Sky Blue", "Light Blue", "Turquoise", "Mint Green", "Emerald Green", "Teal", "Navy Blue"]'
        },
        {
            "role": "user",
            "content": "Generate a color palette for foresta."
        },
        {
            "role": "assistant",
            "content": '["#3F5F6D", "#2F4F4F", "#1B4F5F", "#006633", "#009933", "#33CC33", "#00CC00"], ["Midnight Blue", "Dark Slate Gray", "Dark Turquoise", "Forest Green", "Jungle Green", "Lime Green", "Green"]'
        },
        {
            "role": "user",
            "content": f"Generate a color palette for {text}"
        },
    ]

    completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=500,
            top_p=0.5,
            frequency_penalty=0,
            presence_penalty=0
        )
    print(completion.choices[0].message.content)
    print("-----")
    print(completion)

    result = completion.choices[0].message.content.strip()
    
    colors, names = result.split("], [")

    colors = f"""{colors}]"""
    names = f"""[{names}"""
    
    print(colors)
    print(names)
    
    colors = json.loads(colors)
    names = json.loads(names)
    
    display_colors(colors, names)

## Insert Your Input

In [31]:
get_and_render_prompt("cyberpunk neon light")

 ["#FF0099", "#FF3300", "#FFCC00", "#00FF00", "#0099FF"], ["Neon Magenta", "Neon Orange", "Neon Yellow", "Neon Green", "Neon Blue"]
-----
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " [\"#FF0099\", \"#FF3300\", \"#FFCC00\", \"#00FF00\", \"#0099FF\"], [\"Neon Magenta\", \"Neon Orange\", \"Neon Yellow\", \"Neon Green\", \"Neon Blue\"]"
    }
  ],
  "created": 1684782627,
  "id": "cmpl-7J5DH1K5FIScM3yOeLvBVPnaVR7MI",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 52,
    "prompt_tokens": 280,
    "total_tokens": 332
  }
}
 ["#FF0099", "#FF3300", "#FFCC00", "#00FF00", "#0099FF"]
["Neon Magenta", "Neon Orange", "Neon Yellow", "Neon Green", "Neon Blue"]


<div style="color: #FF0099">Neon Magenta: █████</div> <div style="color: #FF3300">Neon Orange: █████</div> <div style="color: #FFCC00">Neon Yellow: █████</div> <div style="color: #00FF00">Neon Green: █████</div> <div style="color: #0099FF">Neon Blue: █████</div>

In [32]:
get_and_render_prompt_chat("cyberpunk neon light")

["#FF00FF", "#00FFFF", "#FFA500", "#00FF00", "#FF0000", "#FFFF00", "#8A2BE2"], ["Magenta", "Cyan", "Orange", "Lime Green", "Red", "Yellow", "Blue Violet"]
-----
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "[\"#FF00FF\", \"#00FFFF\", \"#FFA500\", \"#00FF00\", \"#FF0000\", \"#FFFF00\", \"#8A2BE2\"], [\"Magenta\", \"Cyan\", \"Orange\", \"Lime Green\", \"Red\", \"Yellow\", \"Blue Violet\"]",
        "role": "assistant"
      }
    }
  ],
  "created": 1684782632,
  "id": "chatcmpl-7J5DMdo95ZCyVQRtYL27Dr13XMVmF",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 62,
    "prompt_tokens": 315,
    "total_tokens": 377
  }
}
["#FF00FF", "#00FFFF", "#FFA500", "#00FF00", "#FF0000", "#FFFF00", "#8A2BE2"]
["Magenta", "Cyan", "Orange", "Lime Green", "Red", "Yellow", "Blue Violet"]


<div style="color: #FF00FF">Magenta: █████</div> <div style="color: #00FFFF">Cyan: █████</div> <div style="color: #FFA500">Orange: █████</div> <div style="color: #00FF00">Lime Green: █████</div> <div style="color: #FF0000">Red: █████</div> <div style="color: #FFFF00">Yellow: █████</div> <div style="color: #8A2BE2">Blue Violet: █████</div>